# Imports

In [1]:
import os
import pickle
import datetime
import inflection

# import numpy    as np
import pandas   as pd
# import seaborn  as sns

# from matplotlib import pyplot as plt

# from IPython.display        import Image
# from IPython.core.display   import HTML

# Metodos Sup

# Load Data

In [2]:
caminho = '../data/raw/'
arquivos = os.listdir(caminho)
lista_arquivos = [arquivo for arquivo in arquivos if arquivo.endswith('.csv')]

In [3]:
lista_arquivos

['azure_ceaec_1ano_ceaec.csv',
 'azure_ceaec_5_set_5_out.csv',
 'azure_editares_1ano.csv',
 'azure_editares_5_set_5_out.csv']

In [14]:
dfs = pd.DataFrame()
for arquivo in lista_arquivos:
    
    df = pd.read_csv( os.path.join( caminho, arquivo ) )
    maquina = arquivo.split('_')[1]
    df['maquina'] = maquina
    
    df = renomear_colunas(df)
    df = mundaca_tipo_date(df)
    
    if dfs.empty:
        dfs = df
    else:
        if maquina in dfs['maquina'].values:
            
            date = dfs.loc[dfs['maquina'] == maquina, 'date']
            filtro = date.max()
            df = df.loc[df['date'] > filtro]
    
    dfs = pd.concat([dfs,df], ignore_index=True)

#df.drop_duplicates(inplace=True)

In [10]:
dfs.shape

(21720, 11)

In [5]:
def renomear_colunas(df):
    
    columns_old = df.columns
    snakecase = lambda x : inflection.underscore(x)
    columns_new = list(map(snakecase,columns_old))
    df.columns = columns_new
    
    return df

In [6]:
def mundaca_tipo_date(df: pd.DataFrame)-> pd.DataFrame:
    
    df['date'] = pd.to_datetime( df['date'] )
    
    return df

# Tratamento

# Export Data

In [11]:
ceaec = pickle.load(open("../src/data/processed/2_0_eda_ceaec.pkl","rb"))
portal = pickle.load(open("../src/data/processed/2_0_eda_portal.pkl","rb"))
ceaec_2 = pickle.load(open("../src/data/processed/2_0_eda_ceaec_2.pkl","rb"))
portal_2 = pickle.load(open("../src/data/processed/2_0_eda_portal_2.pkl","rb"))

In [12]:
portal['maquina'] = "Portal"
ceaec['maquina'] = "CEAEC"
portal_2['maquina'] = "Portal"
ceaec_2['maquina'] = "CEAEC"

filtro_ceaec = ceaec['date'].max()
ceaec_filter = ceaec_2.loc[ceaec_2['date'] > filtro_ceaec]

filtro_portal = portal['date'].max()
portal_filter = portal_2.loc[portal_2['date'] > filtro_portal]

df_pronto_1 = pd.concat([portal,ceaec], ignore_index=True)
df_pronto_2 = pd.concat([portal_filter,ceaec_filter], ignore_index=True)
df_pronto_final = pd.concat([df_pronto_1,df_pronto_2], ignore_index=True)

In [13]:
df_pronto_final.shape

(13515, 18)